In [ ]:
import cv2
import numpy as np
import os
skip = 0
labels = []
facedata = []
datapath = './Data/'
classid = 0
names = {}

def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))


def knn(x,y,querypt,n = 5):
    vals = []
    m = x.shape[0]
    for i in range(m):
        d = dist(x[i],querypt)
        vals.append((d,y[i]))
    vals = sorted(vals)
    vals = vals[:n]
    vals = np.array(vals)

    new_vals = np.unique(vals[1],return_counts = True)
    
    idx = new_vals[1].argmax()
    pred = new_vals[0][idx]
    
    return pred


for fx in os.listdir(datapath):
    if fx.endswith('npy'):
        print("Loaded "+fx)
        names[classid] = fx[:-4]
        data = np.load(datapath+fx)
        facedata.append(data)
        
        target = classid*np.ones(data.shape[0],)
        labels.append(target)
        classid+=1
data_set = np.concatenate(facedata,axis =0 )
label_set = np.concatenate(labels,axis = 0)
#print(data_set.shape)
#print(label_set.shape)


#cap = cv2.VideoCapture(0);
cap = cv2.VideoCapture(0);
face_cascade = cv2.CascadeClassifier('/home/suyashdeshpande/anaconda3/lib/python3.7/site-packages/cv2/data/haarcascade_frontalface_alt.xml')


while True:
    ret, frame = cap.read()
    
    if ret == False:
        continue
    
    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h = face
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        out = knn(data_set,label_set,face_section.flatten())
        
        pred = names[int(out)]
        pred = pred
        cv2.putText(frame,pred,(x-10,y-20),cv2.FONT_HERSHEY_COMPLEX,1,(255,100,0),1,cv2.LINE_AA)
        cv2.rectangle(frame,(x+10,y+10),(x+w+10,y+h+10),(255,100,0),1)
    cv2.imshow('Webcam',frame)
    key = 0XFF & cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
    
        
    
